In [ ]:
import sys
from pymgrid import MicrogridGenerator as mg
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from IPython.core.debugger import set_trace



In [ ]:
import pickle

"""
The buildings mentionned below are specific to the hackathon and are not available in this repo.
You can replace them with any MicroGrid object generated from pymgrid
"""

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    
with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)

buildings = [building_1, building_2, building_3]

In [ ]:
buildings[0].reset()
buildings[1].reset()
buildings[2].reset()

<b> A naive example to get your started can be found below </b>

In [ ]:
def naive_rule_based_strategy(building):
    
    building_data = building.get_updated_values()
    datas=[]

    total_building_cost = []
    total_building_co2 = 0
    while building.done == False:
        load = building_data['load']
        pv = building_data['pv']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']

        p_disc = max(0, min(load-pv, capa_to_dischare, building.battery.p_discharge_max))
        p_char = max(0, min(pv-load, capa_to_charge, building.battery.p_charge_max))

        if load - pv >=  0:

            control_dict = {'battery_charge': 0,
                                'battery_discharge': p_disc,
                                'grid_import': max(0, load-pv-p_disc),
                                'grid_export':0,
                                'pv_consummed': min(pv, load),
                                'genset':1,
                           }


        if load - pv <  0:

            control_dict = {'battery_charge': p_char,
                                'battery_discharge': 0,
                                'grid_import': 0,
                                'grid_export': max(0,pv-load-p_char),#abs(min(load-pv,0)),
                                'pv_consummed': min(pv, load+p_char),
                                'genset':1,
                           }

        building_data = building.run(control_dict)
        total_building_cost.append(building.get_cost())
    
    return total_building_cost

La fonction 

In [ ]:
### Import the Gym environnement with finite States & Actions
import DiscreteEnvironment1 as DiscreteEnvironment

In [ ]:
# fonction qui initie la qtable, notre qtable comporte une variable d'état de plus : le prix de l'electricité
def init_qtable(env, nb_action):
    
    state = []
    Q = {}

    for i in range(-int(env.mg.parameters['PV_rated_power']-1),int(env.mg.parameters['load']+2)):
        
        for k in np.arange(round(env.mg.battery.soc_min,1),round(env.mg.battery.soc_max+0.1,1),0.1):
            for price in ['cheap', 'expensive']:
            
                k = round(k,1)
                state.append((i,k,price))
            #set_trace()

    #Initialize Q(s,a) at zero
    for s in state:

        Q[s] = {}

        for a in range(nb_action):

            Q[s][a] = 0

    return Q

In [ ]:
# fonction décidant de l'action à suivre : aléatoire ou l'action optimal selon la qtable
def espilon_decreasing_greedy(action, epsilon, nb_action):
    
    p = np.random.random()

    if p < (1 - epsilon):
        randomm=0
        return action, randomm

    else: 
        randomm=1
        return np.random.choice(nb_action), randomm

In [ ]:
# fonction argmax de la qtable
def max_dict(d):
# fonction ajustant l'exploration

    max_key = None
    max_val = float('-inf')


    for k,v in d.items():

        if v > max_val:

            max_val = v
            max_key = k

    return max_key, max_val

In [ ]:
# fonction renvoyant la moyenne glissante sur une semaine du coût 
def moyenne_glissante(cost):
    mg = []
    interval = 24*7
    for i in range(len(cost)-interval):
        mg.append(np.mean(cost[i:i+interval]))
    return np.array(mg)

In [ ]:
# fonction ajustant l'exploration
def update_epsilon(epsilon):
    
    epsilon = epsilon - epsilon *0.02
    
    if epsilon < 0.1:
        
        epsilon = 0.1
    
    return epsilon

In [ ]:
def print_welcome(idx):
    
    if idx == 0:
        print("------------------------------------")
        print("|        WELCOME TO PYMGRID        |")
        print("------------------------------------")
    elif idx == 1:
        
        print("t -     STATE  -  ACTION - COST")
        print("================================")


In [ ]:
# construction des qtables 
def training(mg,horizon,nb_action = 5):
    
    env_config={'building':mg}
    building_environment = DiscreteEnvironment.Environment(env_config)
    
    
    Q = init_qtable(building_environment,nb_action)
    #set_trace()
    alpha = 0.2
    nb_state = len(Q)
    nb_episode = 100
    epsilon = 0.99
    gamma = 0.99
    
    record_cost = []
    t0 = time.time()
    t = t0
    print_training = "Training Progressing .   "
    print_welcome(0)
    print("\n")
    
    for e in range(nb_episode+1):
        #alpha = update_alpha(e,nb_episode)
        
        if e == 0:
            value_print="\rEpisode " + str(e) +"/" + str(nb_episode)
            sys.stdout.write(value_print)
            sys.stdout.flush()
        
        else:
            
            now = time.time()
            
            if e == 1 or e == 50 or e == 100 or e == 200 or e == 300 or e == 400  or e == 450  or e == 485 or e == nb_episode-5:
                diff_time_t = now - t 
                total_time = (nb_episode+1) * diff_time_t
            
            now = time.time()
            
            diff_time = now - t0
            time_remaining = total_time - diff_time
            
            t = time.time()
        
        if e % 10 == 0:
        
            if print_training == "Training Progressing .   ":
                
                print_training = "Training Progressing ..  "
                
            elif print_training == "Training Progressing ..  ":
            
                print_training = "Training Progressing ... "
            
            else:
                
                print_training = "Training Progressing .   "

        value_print="\r"+ print_training +"Episode " + str(e) +"/" + str(nb_episode) 
        sys.stdout.write(value_print)
        sys.stdout.flush()
            
        episode_cost = 0
        mg.reset()    
        
        net_load = round(mg.load - mg.pv)
        soc = round(mg.battery.soc,1)
        if mg.grid.price_import > 0.3:
            price = 'expensive'
        else:
            price = 'cheap'
        
       # set_trace()
        
        s = (net_load, soc, price)

        a = max_dict(Q[s])[0]
        a,randomm = espilon_decreasing_greedy(a, epsilon, nb_action)

        for i in range (horizon):

                s_,r,done,info=building_environment.step(a)
                #if building_environment.mg.get_co2()>0:
                #    set_trace()
                
                #r = (-building_environment.mg.get_co2() + r)/2
                episode_cost += r
                if building_environment.mg.grid.price_import > 0.3:
                    s_ = (s_[0],s_[1],'expensive')
                else : 
                    s_ = (s_[0],s_[1],'cheap')
                #s_ = (s_[0],s_[1],price)
                #set_trace()
                a_ = max_dict(Q[s_])[0]
                #a_ = a

                if i == horizon-1:

                    Q[s][a] += alpha*(r - Q[s][a])

                else:

                    old_Q = Q[s][a]
                    target = r + gamma*Q[s_][a_]
                    td_error = target - Q[s][a]
                    Q[s][a] = (1-alpha) * Q[s][a] + alpha * td_error
                    #Q[s][a] = Q[s][a] + alpha * ((r + gamma*Q[s_][a]) - Q[s][a])

                s, a = s_, a_

        epsilon = update_epsilon(epsilon)
        
    return Q

In [ ]:
#fonction executant tout les pas de temps sur une semaine en suivant les règles basiques pour le batiment passé en argument
def naive_rule_for_1_week(building):
    
    building_data = building.get_updated_values()

    total_building_cost = 0
    total_building_co2 = 0
    for i in range(24*7):
        load = building_data['load']
        pv = building_data['pv']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']

        p_disc = max(0, min(load-pv, capa_to_dischare, building.battery.p_discharge_max))
        p_char = max(0, min(pv-load, capa_to_charge, building.battery.p_charge_max))

        if load - pv >=  0:

            control_dict = {'battery_charge': 0,
                                'battery_discharge': p_disc,
                                'grid_import': max(0, load-pv-p_disc),
                                'grid_export':0,
                                'pv_consummed': min(pv, load),
                                'genset':1,
                           }


        if load - pv <  0:

            control_dict = {'battery_charge': p_char,
                                'battery_discharge': 0,
                                'grid_import': 0,
                                'grid_export': max(0,pv-load-p_char),#abs(min(load-pv,0)),
                                'pv_consummed': min(pv, load+p_char),
                                'genset':1,
                           }

        building_data = building.run(control_dict)
        total_building_cost += building.get_cost()
        i+=1
    
    return total_building_cost

In [ ]:
buildings[0].reset()
buildings[1].reset()
buildings[2].reset()

In [ ]:
import time
import json

train_start = time.process_time()
Q1=training(buildings[0],int(8736))
Q2=training(buildings[1],8736,5)
Q3=training(buildings[2],8736,7)
train_end = time.process_time()

In [ ]:
building_environments = [
    DiscreteEnvironment.Environment(env_config={'building':buildings[i]}) for i in range(3)
]
Q=[Q1,Q2,Q3]
test_start2 = time.process_time()

tab_cost_Qlr = [[],[],[]]

for i,building_env in enumerate(building_environments):
    
    obs = building_env.reset(testing=False)
    obs = (obs[0],obs[1],'cheap')
    done = False
    while not done:
        action=max_dict(Q[i][obs])[0]

        obs, reward, done, info = building_env.step(action)
        if building_env.mg.grid.price_import > 0.3:
            obs = (obs[0],obs[1],'expensive')
        else : 
            obs = (obs[0],obs[1],'cheap')
        tab_cost_Qlr[i].append(reward)

test_end2 = time.process_time()

In [ ]:
cost = []
for i in range(3):
    buildings[i].reset(testing = False)
    cost.append(naive_rule_based_strategy(buildings[i]))

In [ ]:
test_start = time.process_time()

building_environments = [
    DiscreteEnvironment.Environment(env_config={'building':buildings[i]}) for i in range(3)
]
Q=[Q1,Q2,Q3]
total_cost = [0,0,0]
total_co2 = [0,0,0]

cost = np.array(cost)

for i,building_env in enumerate(building_environments):
    hour = 0
    
    tab_cost_Qlr[i] = [-i for i in tab_cost_Qlr[i]]
    
    indice_to_switch = np.where(moyenne_glissante(cost[i]) < moyenne_glissante(tab_cost_Qlr[i]))

    
    obs = building_env.reset(testing=True)
    obs = (obs[0],obs[1],'cheap')
    done = False
    while not done:
        if hour in indice_to_switch[0]:
            total_cost[i]+= naive_rule_for_1_week(building_env.mg)
            hour += 24*7
            
        else:
            hour +=1
            action=max_dict(Q[i][obs])[0]
            obs, reward, done, info = building_env.step(action)
            if building_env.mg.grid.price_import > 0.3:
                obs = (obs[0],obs[1],'expensive')
            else : 
                obs = (obs[0],obs[1],'cheap')

            total_cost[i]+=reward
            total_co2[i]+=building_env.mg.get_co2()

test_end = time.process_time()

In [ ]:
#total_cost *=-1
total_cost

In [ ]:
frug = train_end + test_end +test_end2 - (train_start + test_start +test_start)

In [ ]:
final_results = {
    "building_1_performance" : -total_cost[0],
    "building_2_performance" : -total_cost[1],
    "building_3_performance" : -total_cost[2],
    "frugality" : frug,
}
print(final_results)

L'implémentation d'un algo hybride nous a été motivé par le graphique ci-dessous, on constate bien que pour certaine semaine (notament en fin d'année) le qlearning est bien moins efficace que des règles tout à fait basique.

In [ ]:
plt.plot(moyenne_glissante(cost_plot) < moyenne_glissante(tab_cost_Qlr_plot))